In [1]:
# coding=utf-8
from __future__ import absolute_import, division, print_function

import os
import random
import json
import sys
import re
import numpy as np
import collections

import torch
from torch.utils.data import (DataLoader, TensorDataset)

from IR.mix_LM import Config, BertMultiTask_ICT as BertMultiTask
#from IR.transformer import Config, BertMultiTask_ICT as BertMultiTask

import transformers
from transformers.data.processors.squad import SquadV2Processor
from transformers import squad_convert_examples_to_features

In [2]:
batch_size = 16
max_seq_length = 384
max_query_length = 64
doc_stride = 128
model_file = 'IR/model.bin'
#config_file = 'IR/transformer_config.json'
config_file = 'IR/mix_LM_config.json'

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda")
n_gpu = torch.cuda.device_count()

In [3]:
def set_seed():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)

def _check_is_max_context(doc_spans, cur_span_index, position):
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

def check_answer(sim, contexts, answer, valid_rank=100) :
    "get accuracy based on given similarity score"
    sim = np.flip(np.argsort(sim, axis=1), axis=1)[:, :valid_rank]
    hits = []
    for a, s in zip(answer, sim) :
        hit = []
        for i in s :
            hit.append((a in contexts[i]))
        hits.append(hit)
    hits = np.array(hits)
    true_hit = np.zeros(hits.shape[0])!=0
    hit_rates = []
    for i in range(valid_rank) :
        true_hit = (hits[:, i].reshape(-1))|true_hit
        hit_rates.append(round((np.sum(true_hit)/len(true_hit))*100, 2))
        print("{} rank : {}".format(i+1, hit_rates[-1]))
    print('')
    return hit_rates[0]

In [4]:
set_seed()
tokenizer = transformers.ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

config = Config.from_json_file(config_file)
config.vocab_size = tokenizer.vocab_size
config.max_seq_length = max_seq_length

model = BertMultiTask(config, train_mode='ict')
model.load_state_dict(torch.load(model_file))
model.to(device)

input_file = 'data/KorQuAD_v1.0_dev.json'
processor = SquadV2Processor()
input_a = input_file.split('/')
examples = processor.get_train_examples(''.join(input_a[:-1]), filename=input_a[-1])


para_d = ""
c_data, q_data, a_data = [], [], []
for ex in examples:
    ' '.join(ex.doc_tokens)
    if para_d != ex.context_text:  # 문서가 바뀌었는지 체크하여 문서 번호 체크
        para_d = ex.context_text
        c_data.append([para_d, tokenizer.tokenize(para_d)])
    
    q_text = ex.question_text
    q_data.append([q_text, tokenizer.tokenize(q_text)])
    
    a_text = ex.answer_text
    a_data.append(a_text)


100%|██████████| 140/140 [00:01<00:00, 78.65it/s]


In [5]:
c_features = []
for cd, ct in c_data:
    
    all_doc_tokens = [t for t in ct]
    max_tokens_for_doc = max_seq_length - 2
    
    _DocSpan = collections.namedtuple(
            "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_tokens_for_doc:
            length = max_tokens_for_doc
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
            break
        start_offset += min(length, doc_stride)
    
    for (doc_span_index, doc_span) in enumerate(doc_spans):
        c_tokens = []
        c_tokens.append(tokenizer.cls_token)
        token_to_orig_map = {}
        token_is_max_context = {}
        for i in range(doc_span.length):
            split_token_index = doc_span.start + i
            token_to_orig_map[len(c_tokens)] = split_token_index

            is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                   split_token_index)
            token_is_max_context[len(c_tokens)] = is_max_context
            c_tokens.append(all_doc_tokens[split_token_index])
        c_tokens.append(tokenizer.sep_token)
    
        c_input_ids = [tokenizer.convert_tokens_to_ids(t) for t in c_tokens]
        c_input_mask = [1] * len(c_input_ids)
    
        while len(c_input_ids) < max_seq_length:
            c_input_ids.append(0)
            c_input_mask.append(0)
        
        assert len(c_input_ids) == max_seq_length
        assert len(c_input_mask) == max_seq_length
        
        c_features.append({
            "text" : cd,
            "input_ids" : c_input_ids, 
            "attention_mask" : c_input_mask,
        })

In [6]:
q_features = []
for qd, qt in q_data:
    
    max_tokens_for_que = max_query_length - 2

    if len(qt) > max_tokens_for_que:
        qt = qt[0:max_tokens_for_que]
    
    q_tokens, q_segment_ids = [], []
    q_tokens.append(tokenizer.cls_token)
    for i in range(len(qt)):
        q_tokens.append(qt[i])
    q_tokens.append(tokenizer.sep_token)
    
    q_input_ids = [tokenizer.convert_tokens_to_ids(t) for t in q_tokens]
    q_input_mask = [1] * len(q_input_ids)
    
    while len(q_input_ids) < max_query_length:
        q_input_ids.append(0)
        q_input_mask.append(0)
        
    assert len(q_input_ids) == max_query_length
    assert len(q_input_mask) == max_query_length

    q_features.append({
        "text" : qd,
        "input_ids" : q_input_ids, 
        "attention_mask" : q_input_mask,
    })

In [7]:
q_input_ids = torch.tensor([f['input_ids'] for f in q_features], dtype=torch.long)
q_input_mask = torch.tensor([f['attention_mask'] for f in q_features], dtype=torch.long)
q_dataset = TensorDataset(q_input_ids, q_input_mask)

c_input_ids = torch.tensor([f['input_ids'] for f in c_features], dtype=torch.long)
c_input_mask = torch.tensor([f['attention_mask'] for f in c_features], dtype=torch.long)
c_dataset = TensorDataset(c_input_ids, c_input_mask)

In [8]:
c_dataloader = DataLoader(c_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
q_dataloader = DataLoader(q_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [9]:
n_ans_ids = [tokenizer.cls_token_id, tokenizer.sep_token_id]
n_ans_mask = [1] * len(n_ans_ids)
while len(n_ans_ids) < max_seq_length:
    n_ans_ids.append(0)
    n_ans_mask.append(0)

context_embedding = []
question_embedding = []
model.eval()
with torch.no_grad():
    
    n_ans = torch.tensor([n_ans_ids, n_ans_mask], dtype=torch.long)
    n_ans = tuple(t.to(device) for t in n_ans)
    n_ans_ids, n_ans_mask = n_ans
    c_encode = model.bert(input_ids = n_ans_ids.unsqueeze(0), attention_mask = n_ans_mask.unsqueeze(0))
    c_encode_pooled = model.decoder(c_encode)
    context_embedding.append(c_encode_pooled)
    
    for step, batch in enumerate(c_dataloader):
        batch = tuple(t.to(device) for t in batch)
        c_input_ids, c_input_mask = batch
        c_encode = model.bert(input_ids = c_input_ids, attention_mask = c_input_mask)
        c_encode_pooled = model.decoder(c_encode)
        context_embedding.append(c_encode_pooled)
    context_embedding = torch.cat(context_embedding, 0)
    for step, batch in enumerate(q_dataloader):
        batch = tuple(t.to(device) for t in batch)
        q_input_ids, q_input_mask = batch
        q_encode = model.bert(input_ids = q_input_ids, attention_mask = q_input_mask)
        q_encode_pooled = model.decoder(q_encode)
        question_embedding.append(q_encode_pooled)
    question_embedding = torch.cat(question_embedding, 0)

In [10]:
print('IR 성능')
context_text = ['']
for i in c_features:
    context_text.append(i["text"])

IR 성능


In [11]:
semantic_sim = torch.matmul(question_embedding, context_embedding.t()).detach().cpu().numpy()
check_answer(semantic_sim, context_text, a_data, 10)

1 rank : 65.4
2 rank : 77.85
3 rank : 83.82
4 rank : 86.77
5 rank : 88.9
6 rank : 90.51
7 rank : 91.6
8 rank : 92.54
9 rank : 93.26
10 rank : 93.8



65.4

In [12]:
# 샘플링 실험
index = 5
num_topk = 10
print(q_features[index]["text"])
cs = torch.matmul(question_embedding[index], context_embedding.t())
print(torch.topk(cs, k=num_topk, dim=-1)[0].tolist())
topk = torch.topk(cs, k=num_topk, dim=-1)[1].tolist()
for i in topk:
    print(c_features[int(i-1)]["text"])
    print()

1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?
[91.27508544921875, 89.81043243408203, 87.6229019165039, 87.38345336914062, 85.51984405517578, 85.10320281982422, 84.95995330810547, 84.68521118164062, 84.68035125732422, 84.18185424804688]
박정희 정부로부터 질산 테러 등의 탄압을 받았다고 주장했다. 1979년 10월에는 YH 무역 여공 농성 사건 이후 타임과의 인터뷰에서 미국에 박정희 정권에 대한 지지를 철회할 것을 주장하였다. 유신정권은 이 발언을 문제삼아 의원직 제명 파동을 일으켜 부마항쟁을 촉발했다. 1983년에는 5.18 광주 민주화 운동 기념일을 기해 23일 동안 단식투쟁에 돌입했다. 6월 민주 항쟁 이후 통일민주당 총재로 민주화추진협의회를 구성해 민주진영을 구축했다. 1986년 대통령 직선제 개헌 1천만 서명운동을 전개하였다. 1990년 민주정의당-통일민주당-신민주공화당 3당 합당을 선언하여 민주자유당 대표최고위원으로 추대되었다. 1993년 제14대 대통령에 취임하며 32년만에 군사 정권의 마침표를 찍었고, 문민 정부를 열었다. 예술인과 작가들의 반정부와 사회비판을 허용하였다.

1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15

In [13]:
print('TF-IDF 성능')
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

ques_text = []
for i in q_data:
    ques_text.append(i[0])
    
#answ_text = []
#for i in q_data:
#    answ_text.append(i[2])

context_text = []
for i in c_data:
    context_text.append(i[0])

#ques_text = []
#for i in q_features:
#    ques_text.append(i[0])
    
ngram = 2
tfidf = TfidfVectorizer(analyzer=str.split
                            , encoding="utf-8"
                            , stop_words="korean"
                            , ngram_range=(1, 2))

tfidf_context = tfidf.fit_transform([context for context in context_text])
tfidf_question = tfidf.transform(ques_text)
tfidf_sim = cosine_similarity(tfidf_question, tfidf_context)
check_answer(tfidf_sim, context_text, a_data, 100)

TF-IDF 성능
1 rank : 71.35
2 rank : 80.38
3 rank : 84.29
4 rank : 86.32
5 rank : 87.81
6 rank : 88.74
7 rank : 89.42
8 rank : 90.04
9 rank : 90.58
10 rank : 90.84
11 rank : 91.27
12 rank : 91.51
13 rank : 91.79
14 rank : 91.95
15 rank : 92.15
16 rank : 92.29
17 rank : 92.43
18 rank : 92.55
19 rank : 92.62
20 rank : 92.74
21 rank : 92.85
22 rank : 92.97
23 rank : 93.09
24 rank : 93.25
25 rank : 93.37
26 rank : 93.45
27 rank : 93.51
28 rank : 93.56
29 rank : 93.64
30 rank : 93.78
31 rank : 93.82
32 rank : 93.9
33 rank : 93.94
34 rank : 93.99
35 rank : 94.02
36 rank : 94.02
37 rank : 94.09
38 rank : 94.09
39 rank : 94.16
40 rank : 94.2
41 rank : 94.22
42 rank : 94.27
43 rank : 94.34
44 rank : 94.42
45 rank : 94.42
46 rank : 94.46
47 rank : 94.49
48 rank : 94.53
49 rank : 94.54
50 rank : 94.6
51 rank : 94.61
52 rank : 94.61
53 rank : 94.61
54 rank : 94.7
55 rank : 94.77
56 rank : 94.8
57 rank : 94.82
58 rank : 94.82
59 rank : 94.86
60 rank : 94.89
61 rank : 94.89
62 rank : 94.91
63 rank : 94

71.35